# Pokémon Battle Game
![Pokemon Battle](https://media3.giphy.com/media/v1.Y2lkPTc5MGI3NjExNGY2cmNoaWE1ZTA5NHBtdmljc3Qxamt1NzliNWgzYWxwNm54M2pkZyZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/FAxPIii1As4UpqmEkx/giphy.gif)

## This notebook will guide you through building an interactive Pokémon battle game using the PokeAPI.

### Features:
- **Fetch Pokémon data (name, abilities, moves, types) using the `requests` library**
- **Build a Pokémon team and battle gym leaders**
- **Simulated turn-based battle system**
- **Unit testing with `unittest`**
- **Over 1118 Pokémon available**
- **20 Pokémon types with type effectiveness applied**
- **Simulated turn-based 6v6 battle system**


In [22]:
import requests
import random
import unittest

### Step 1: Define the Pokedex Class
#### The Pokedex class fetches Pokémon details using the PokeAPI and stores them.

In [23]:
class Pokedex:
    """Class to fetch and store Pokémon data."""
    BASE_URL = "https://pokeapi.co/api/v2/pokemon/"
    TYPE_EFFECTIVENESS = {
        "normal": {"rock": 0.5, "ghost": 0.0, "steel": 0.5},
        "fire": {"grass": 2.0, "ice": 2.0, "bug": 2.0, "steel": 2.0, "fire": 0.5, "water": 0.5, "rock": 0.5, "dragon": 0.5},
        "water": {"fire": 2.0, "ground": 2.0, "rock": 2.0, "water": 0.5, "grass": 0.5, "dragon": 0.5},
        "grass": {"water": 2.0, "ground": 2.0, "rock": 2.0, "fire": 0.5, "grass": 0.5, "poison": 0.5, "flying": 0.5, "bug": 0.5, "dragon": 0.5, "steel": 0.5},
        "electric": {"water": 2.0, "flying": 2.0, "electric": 0.5, "grass": 0.5, "dragon": 0.5, "ground": 0.0},
        "ice": {"grass": 2.0, "ground": 2.0, "flying": 2.0, "dragon": 2.0, "fire": 0.5, "water": 0.5, "ice": 0.5, "steel": 0.5},
        "fighting": {"normal": 2.0, "ice": 2.0, "rock": 2.0, "dark": 2.0, "steel": 2.0, "poison": 0.5, "flying": 0.5, "psychic": 0.5, "bug": 0.5, "fairy": 0.5, "ghost": 0.0},
        "poison": {"grass": 2.0, "fairy": 2.0, "poison": 0.5, "ground": 0.5, "rock": 0.5, "ghost": 0.5, "steel": 0.0},
        "ground": {"fire": 2.0, "electric": 2.0, "poison": 2.0, "rock": 2.0, "steel": 2.0, "grass": 0.5, "bug": 0.5, "flying": 0.0},
        "flying": {"grass": 2.0, "fighting": 2.0, "bug": 2.0, "electric": 0.5, "rock": 0.5, "steel": 0.5},
        "psychic": {"fighting": 2.0, "poison": 2.0, "psychic": 0.5, "steel": 0.5, "dark": 0.0},
        "bug": {"grass": 2.0, "psychic": 2.0, "dark": 2.0, "fire": 0.5, "fighting": 0.5, "poison": 0.5, "flying": 0.5, "ghost": 0.5, "steel": 0.5, "fairy": 0.5},
        "rock": {"fire": 2.0, "ice": 2.0, "flying": 2.0, "bug": 2.0, "fighting": 0.5, "ground": 0.5, "steel": 0.5},
        "ghost": {"psychic": 2.0, "ghost": 2.0, "normal": 0.0, "dark": 0.5},
        "dragon": {"dragon": 2.0, "steel": 0.5, "fairy": 0.0},
        "dark": {"psychic": 2.0, "ghost": 2.0, "fighting": 0.5, "dark": 0.5, "fairy": 0.5},
        "steel": {"ice": 2.0, "rock": 2.0, "fairy": 2.0, "fire": 0.5, "water": 0.5, "electric": 0.5, "steel": 0.5},
        "fairy": {"fighting": 2.0, "dragon": 2.0, "dark": 2.0, "fire": 0.5, "poison": 0.5, "steel": 0.5}
    }
    
    def __init__(self):
        self.pokemon_data = {}
    
    def get_pokemon(self, name):
        """Fetch Pokémon details from PokeAPI with error handling."""
        if name in self.pokemon_data:
            return self.pokemon_data[name]
        
        try:
            response = requests.get(f"{self.BASE_URL}{name.lower()}", timeout=5)
            response.raise_for_status()
            data = response.json()
            pokemon_info = {
                "name": data["name"],
                "abilities": [ability["ability"]["name"] for ability in data["abilities"]],
                "moves": [move["move"]["name"] for move in data["moves"]][:4],
                "types": [ptype["type"]["name"] for ptype in data["types"]],
                "hp": random.randint(50, 100)  # Randomized HP for gameplay
            }
            self.pokemon_data[name] = pokemon_info
            return pokemon_info
        except requests.exceptions.RequestException as e:
            print(f"Error fetching Pokémon: {e}")
            return None

### Step 2: Define the Battle System
#### This class manages turn-based 6v6 battles between Pokémon.

In [24]:
class PokemonBattle:
    """Class to manage Pokémon battles."""
    def __init__(self, trainer, gym_leader):
        self.trainer = trainer
        self.gym_leader = gym_leader
    
    def type_effectiveness(self, attack_types, defend_types):
        """Calculate type effectiveness multiplier."""
        multiplier = 1.0
        for attack_type in attack_types:
            for defend_type in defend_types:
                multiplier *= Pokedex.TYPE_EFFECTIVENESS.get(attack_type, {}).get(defend_type, 1.0)
        return multiplier
    
    def attack(self, attacker, defender):
        """Simulate an attack move with type effectiveness."""
        move = random.choice(attacker["moves"])
        effectiveness = self.type_effectiveness(attacker["types"], defender["types"])
        damage = int(random.randint(10, 30) * effectiveness)
        defender["hp"] -= damage
        print(f"{attacker['name']} used {move}! It dealt {damage} damage.")
        if defender["hp"] <= 0:
            print(f"{defender['name']} fainted!")
            return True
        return False
    
    def start_battle(self):
        """Start the 6v6 battle between the trainer and the gym leader."""
        print(f"Battle Start: {self.trainer.name} vs {self.gym_leader.name}!")
        
        for trainer_pokemon, gym_pokemon in zip(self.trainer.team, self.gym_leader.team):
            print(f"{trainer_pokemon['name']} VS {gym_pokemon['name']}")
            while trainer_pokemon["hp"] > 0 and gym_pokemon["hp"] > 0:
                if self.attack(trainer_pokemon, gym_pokemon):
                    break
                if self.attack(gym_pokemon, trainer_pokemon):
                    break
        
        if any(pokemon["hp"] > 0 for pokemon in self.trainer.team):
            print(f"{self.trainer.name} wins!")
        else:
            print(f"{self.gym_leader.name} wins!")

### Step 3: Define Trainers and Gym Leaders
#### Trainers can catch Pokémon and battle gym leaders.

In [25]:
class Trainer:
    """Class for a Pokémon trainer who can build a team and battle gym leaders."""
    def __init__(self, name):
        self.name = name
        self.team = []
    
    def catch_pokemon(self, pokedex, pokemon_name):
        """Add a Pokémon to the trainer's team."""
        if len(self.team) < 6:
            pokemon = pokedex.get_pokemon(pokemon_name)
            if pokemon:
                self.team.append(pokemon)
                print(f"{pokemon_name} added to your team!")
            else:
                print("Could not find Pokémon.")
        else:
            print("Your team is full!")
    
    def show_team(self):
        """Display the trainer's Pokémon team."""
        if self.team:
            print(f"{self.name}'s Team:")
            for pokemon in self.team:
                print(f"- {pokemon['name']} (HP: {pokemon['hp']})")
        else:
            print("You have no Pokémon in your team.")

class GymLeader(Trainer):
    """Class representing a Gym Leader with a fixed Pokémon team."""
    pass

### Step 4: Unit Testing
#### We use `unittest` to verify the correctness of our Pokedex implementation.

In [26]:
class TestPokedex(unittest.TestCase):
    def setUp(self):
        self.pokedex = Pokedex()
    
    def test_fetch_pokemon(self):
        pokemon = self.pokedex.get_pokemon("pikachu")
        self.assertIsNotNone(pokemon)
        self.assertEqual(pokemon["name"], "pikachu")
    
    def test_fetch_invalid_pokemon(self):
        pokemon = self.pokedex.get_pokemon("invalidpokemon")
        self.assertIsNone(pokemon)
    
    def test_cache_pokemon(self):
        self.pokedex.get_pokemon("pikachu")
        self.assertIn("pikachu", self.pokedex.pokemon_data)
    
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.130s

OK


Error fetching Pokémon: 404 Client Error: Not Found for url: https://pokeapi.co/api/v2/pokemon/invalidpokemon


### Step 5: Play the Game
#### The trainer can catch Pokémon and battle a gym leader.

In [27]:
if __name__ == "__main__":
    pokedex = Pokedex()
    trainer_name = input("Enter your name: ")
    trainer = Trainer(trainer_name)
    
    while len(trainer.team) < 6:
        pokemon_name = input("Catch a Pokémon: ")
        trainer.catch_pokemon(pokedex, pokemon_name)
    
    trainer.show_team()
    
    gym_leader = GymLeader("Brock")
    gym_leader.team = [pokedex.get_pokemon(pokemon) for pokemon in ["onix", "geodude", "graveler", "golem", "kabuto", "omastar"]]
    
    battle = PokemonBattle(trainer, gym_leader)
    battle.start_battle()


Enter your name:  Mori
Catch a Pokémon:  Gengar


Gengar added to your team!


Catch a Pokémon:  Snorlax


Snorlax added to your team!


Catch a Pokémon:  Lucario


...

Rayquaza added to your team!


Catch a Pokémon:  Darkrai


Darkrai added to your team!


Catch a Pokémon:  Pikachu


Pikachu added to your team!
Mori's Team:
- gengar (HP: 74)
- snorlax (HP: 79)
- lucario (HP: 64)
- rayquaza (HP: 93)
- darkrai (HP: 82)
- pikachu (HP: 88)
Battle Start: Mori vs Brock!
gengar VS onix
gengar used thunder-punch! It dealt 6 damage.
onix used headbutt! It dealt 40 damage.
gengar used mega-punch! It dealt 7 damage.
onix used tackle! It dealt 26 damage.
gengar used thunder-punch! It dealt 5 damage.
onix used headbutt! It dealt 52 damage.
gengar fainted!
snorlax VS geodude
snorlax used ice-punch! It dealt 6 damage.
geodude used thunder-punch! It dealt 18 damage.
snorlax used fire-punch! It dealt 7 damage.
geodude used thunder-punch! It dealt 22 damage.
snorlax used mega-punch! It dealt 7 damage.
geodude used mega-punch! It dealt 19 damage.
snorlax used fire-punch! It dealt 12 damage.
geodude used sand-attack! It dealt 18 damage.
snorlax used ice-punch! It dealt 14 damage.
geodude used fire-punch! It dealt 21 damage.
snorlax fainted!
lucario VS graveler
lucario used swords-danc